In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, minmax_scale
from datetime import datetime

sys.path.append('../../holodecml/')

from data import load_scaled_datasets, load_raw_datasets, calc_z_bins

In [2]:
path_data = "/glade/p/cisl/aiml/ai4ess_hackathon/holodec/"
num_particles = "large"
output_cols = ["x", "y", "z", "d", "hid"]
scaler_out = MinMaxScaler() #use softmax to bind outputs from 0 to 1
subset = False
num_z_bins = False
mass = False


In [3]:
inputs, outputs = load_raw_datasets(path_data,
                                    num_particles, 
                                    "train",
                                    output_cols,
                                    subset)


In [4]:
def make_template(df, num_images):
    max_particles = df['hid'].value_counts().max()
    size = (max_particles * num_images, 1) 
    x = np.random.uniform(low=df['x'].min(), high=df['x'].max(), size=size)
    y = np.random.uniform(low=df['y'].min(), high=df['y'].max(), size=size)
    z = np.random.uniform(low=df['z'].min(), high=df['z'].max(), size=size)
    d = np.random.uniform(low=df['d'].min(), high=df['d'].max(), size=size)
    prob = np.zeros(d.shape)
    template = np.hstack((x, y ,z ,d ,prob))
    template = template.reshape((num_images, max_particles, -1))
    return template

def outputs_3d(outputs, num_images):
    outputs_array = make_template(outputs, num_images)
    print(outputs_array.shape)
    for hid in outputs["hid"].unique():
        outputs_hid = outputs.loc[outputs['hid'] == hid].to_numpy()
        outputs_hid[:, -1] = 1
        outputs_array[int(hid-1), :outputs_hid.shape[0], :] = outputs_hid
    return outputs_array


In [5]:
outputs

,x,y,z,d,hid
particle,,,,,
0,-159.248978,992.123230,39779.312500,14.298387,1.0
1,1312.140991,309.901398,24792.720703,38.611160,1.0
2,-1534.479980,-323.795258,119003.070312,55.779282,1.0
3,1163.250000,393.965179,126016.921875,63.675018,1.0
4,-1425.947876,-452.177429,119391.359375,29.566635,1.0
...,...,...,...,...,...
373847,-1273.068970,467.297272,119432.562500,37.348354,5000.0
373848,-271.437988,683.952148,154884.484375,56.189152,5000.0
373849,-1498.370972,655.821716,54045.691406,101.987999,5000.0


In [6]:
template = make_template(outputs, inputs.shape[0])

In [7]:
if inputs.shape[0] != outputs.shape[0]:
    print("false")


false


In [8]:
scale_cols = [x for x in output_cols if x != 'hid']
outputs[scale_cols] = scaler_out.fit_transform(outputs[scale_cols])
outputs


,x,y,z,d,hid
particle,,,,,
0,0.455544,0.920126,0.179021,0.061103,1.0
1,0.870134,0.631662,0.074948,0.165479,1.0
2,0.068048,0.363717,0.729188,0.239183,1.0
3,0.828181,0.667207,0.777896,0.273080,1.0
4,0.098629,0.309433,0.731885,0.126651,1.0
...,...,...,...,...,...
373847,0.141705,0.698214,0.732171,0.160058,5000.0
373848,0.423932,0.789822,0.978365,0.240942,5000.0
373849,0.078223,0.777928,0.278094,0.437559,5000.0


In [9]:
outputs_3d = outputs_3d(outputs, inputs.shape[0])
outputs_3d.shape


(5000, 100, 5)


(5000, 100, 5)

In [11]:
outputs_3d[0][0]

array([0.45554373, 0.92012572, 0.17902146, 0.06110322, 1.        ])

In [15]:
outputs_3d[0][58]

array([0.99451869, 0.124138  , 0.13986373, 0.73327381, 0.        ])